## 생존여부 예측모델 만들기
### 학습용 데이터 (X_train, y_train)을 이용하여 생존 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측값을 다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 accuracy 평가지표에 따라 채점)

(가) 제공 데이터 목록
- y_train: 생존여부(학습용)
- X_trian, X_test : 승객 정보 (학습용 및 평가용)

(나) 데이터 형식 및 내용
- y_trian (712명 데이터)

**시험환경 세팅은 예시문제와 동일한 형태의 X_train, y_train, X_test 데이터를 만들기 위함임**

### 유의사항
- 성능이 우수한 예측모형을 구축하기 위해서는 적절한 데이터 전처리, 피처엔지니어링, 분류알고리즘, 하이퍼파라미터 튜닝, 모형 앙상블 등이 수반되어야 한다.
- 수험번호.csv파일이 만들어지도록 코드를 제출한다.
- 제출한 모델의 성능은 accuracy로 평가함

csv 출력형태

![image.png](attachment:de1920de-121e-47c3-a61f-e905386713bf.png)

## [참고]작업형2 문구
- 출력을 원하실 경우 print() 함수 활용
- 예시) print(df.head())
- getcwd(), chdir() 등 작업 폴더 설정 불필요
- 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

### 데이터 파일 읽기 예제
- import pandas as pd
- X_test = pd.read_csv("data/X_test.csv")
- X_train = pd.read_csv("data/X_train.csv")
- y_train = pd.read_csv("data/y_train.csv")

### 사용자 코딩

### 답안 제출 참고
- 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
- pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)

In [1]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=2021)
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[id_name, target])
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[id_name, target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("../input/titanic/train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, 
                                                  target='Survived', 
                                                  id_name='PassengerId')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Start
### 라이브러리 및 데이터 불러오기


In [1]:
### 라이브러리 및 데이터 불러오기
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [1]:
X_train.head()

In [1]:
X_test.head()

In [1]:
y_train.head()

### 전처리 및 EDA

In [1]:
X_train.info()

In [1]:
X_train.describe()

In [1]:
X_test.info()

Age, Cabin, Embarked에 결측치 존재

결측치가 너무 많은 Cabin은 제거

In [1]:
temp = X_train.copy()
temp['target'] = y_train.iloc[:,1]

In [1]:
temp.head()

In [1]:
temp['Embarked'].describe()

In [1]:
temp['Embarked'] = temp['Embarked'].fillna('S') # 최빈값 대체

In [1]:
temp.groupby(['Pclass', 'Sex']).median()['Age'] # 클래스별 차이가 있으므로 해당 중위값으로 대체

In [1]:
temp.loc[(temp['Pclass']==1)&(temp['Sex']=='female')&(temp['Age'].isna()),'Age'] = 35
temp.loc[(temp['Pclass']==1)&(temp['Sex']=='male')&(temp['Age'].isna()),'Age'] = 38
temp.loc[(temp['Pclass']==2)&(temp['Sex']=='female')&(temp['Age'].isna()),'Age'] = 30
temp.loc[(temp['Pclass']==2)&(temp['Sex']=='male')&(temp['Age'].isna()),'Age'] = 30
temp.loc[(temp['Pclass']==3)&(temp['Sex']=='female')&(temp['Age'].isna()),'Age'] = 19
temp.loc[(temp['Pclass']==3)&(temp['Sex']=='male')&(temp['Age'].isna()),'Age'] = 25

In [1]:
temp = pd.concat([temp, pd.get_dummies(temp.Sex).iloc[:,:-1]], axis = 1)
temp = pd.concat([temp, pd.get_dummies(temp.Embarked).iloc[:,:-1]], axis = 1)

temp.drop(['Name', 'Cabin', 'Sex', 'Embarked', 'Ticket'],axis=1, inplace=True)

In [1]:
X_test['Embarked'] = X_test['Embarked'].fillna('S')

X_test.loc[(X_test['Pclass']==1)&(X_test['Sex']=='female')&(X_test['Age'].isna()),'Age'] = 35
X_test.loc[(X_test['Pclass']==1)&(X_test['Sex']=='male')&(X_test['Age'].isna()),'Age'] = 38
X_test.loc[(X_test['Pclass']==2)&(X_test['Sex']=='female')&(X_test['Age'].isna()),'Age'] = 30
X_test.loc[(X_test['Pclass']==2)&(X_test['Sex']=='male')&(X_test['Age'].isna()),'Age'] = 30
X_test.loc[(X_test['Pclass']==3)&(X_test['Sex']=='female')&(X_test['Age'].isna()),'Age'] = 19
X_test.loc[(X_test['Pclass']==3)&(X_test['Sex']=='male')&(X_test['Age'].isna()),'Age'] = 25

X_test = pd.concat([X_test, pd.get_dummies(X_test.Sex).iloc[:,:-1]], axis = 1)
X_test = pd.concat([X_test, pd.get_dummies(X_test.Embarked).iloc[:,:-1]], axis = 1)

X_test.drop(['Name', 'Cabin', 'Sex', 'Embarked' ,'Ticket'], axis=1, inplace=True)

In [1]:
temp.info()

In [1]:
X_test.info()

In [1]:
temp.describe()

In [1]:
X_test.describe()

In [1]:
temp.groupby('target').mean()

- 사망 : 남성, 클래스가 낮을수록, 요금이 쌀수록 사망비율이 높다.
- 생존 : 여성, 클래스가 높을수록, 요금이 비쌀수록 생존비율이 높다.

In [1]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBRFClassifier, XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [1]:
X_tra, X_val, y_tra, y_val = train_test_split(temp.drop('target',axis=1), temp.target, test_size=0.2, random_state=42)

In [1]:
list_ml = [AdaBoostClassifier(random_state=42), GradientBoostingClassifier(random_state=42), 
           RandomForestClassifier(random_state=42), ExtraTreesClassifier(random_state=42),
           SVC(random_state=42), DecisionTreeClassifier(random_state=42), XGBRFClassifier(random_state=42), XGBClassifier(random_state=42)]

In [1]:
for i in range(len(list_ml)):
    clf = list_ml[i]
    clf.fit(X_tra, y_tra)
    print(list_ml[i],':', clf.score(X_val, y_val))

- GradientBoostingClassifier(random_state=42) : 0.8671328671328671
- RandomForestClassifier(random_state=42) : 0.8391608391608392
- XGBRFClassifier() : 0.8461538461538461

상위 3개의 모델의 최적 하이퍼파라미터를 찾기로 한다.

In [1]:
X_train = temp.drop('target',axis=1)
y_train = temp.target

In [1]:
parameters = {'learning_rate':[0.01, 0.005], 'n_estimators':[100,300,500]}
clf = GridSearchCV(GradientBoostingClassifier(random_state=42), parameters)
clf.fit(X_train, y_train)
print(clf.best_estimator_)
print(clf.best_score_)

In [1]:
parameters = {'max_depth':[-1,5,10], 'n_estimators':[100,300,500]}
clf = GridSearchCV(RandomForestClassifier(random_state=42), parameters)
clf.fit(X_train, y_train)
print(clf.best_estimator_)
print(clf.best_score_)

In [1]:
parameters = {'max_depth':[-1,5,10], 'learning_rate':[0.01, 0.005], 'n_estimators':[100,300,500]}
clf = GridSearchCV(XGBRFClassifier(random_state=42), parameters)
clf.fit(X_train, y_train)
print(clf.best_estimator_)
print(clf.best_score_)

0.8483305426967398으로 가장 점수가 높은 RandomForestClassifier(max_depth=10, n_estimators=500, random_state=42)을 최종모델 선정

In [1]:
clf = RandomForestClassifier(max_depth=10, n_estimators=500, random_state=42)
clf.fit(X_train, y_train)
pred = y_test.copy()
pred.iloc[:,1] = clf.predict(X_test)

In [1]:
print('최종 스코어 :', accuracy_score(pred.iloc[:,1], y_test.iloc[:,1]))

In [1]:
# pred.to_csv('titanic.csv', index = False)

## kaggle code

## 라이브러리 및 데이터 불러오기

In [1]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=2021)
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[id_name, target])
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[id_name, target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("../input/titanic/train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, 
                                                  target='Survived', 
                                                  id_name='PassengerId')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [1]:
# 라이브러리 불러오기
import pandas as pd

In [1]:
# 데이터 불러오기 (생략)
X_train.shape, y_train.shape, X_test.shape

## EDA

In [1]:
X_train.head()

In [1]:
# float64(2), int64(3), object(5)
X_train.info()

In [1]:
y_train.head()

In [1]:
# 생존 비율
y_train['Survived'].value_counts()

## 데이터 전처리

In [1]:
y = y_train["Survived"]

# sex만 원핫인코딩 됨
features = ["Pclass", "Sex", "SibSp", "Parch"]
X_train = pd.get_dummies(X_train[features])
X_test = pd.get_dummies(X_test[features])

In [1]:
X_train.shape, X_test.shape

In [1]:
X_train.head()

## 모델 및 평가

In [1]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200, max_depth=7, random_state=2021)
model.fit(X_train, y)
predictions = model.predict(X_test)

In [1]:
model.score(X_train, y)

In [1]:
output = pd.DataFrame({'PassengerId': y_test.PassengerId, 'Survived': predictions})
output.head()

In [1]:
# 수험번호.csv로 출력
output.to_csv('1234567.csv', index=False)

## 결과 체점 (수험자는 알 수 없는 부분임)

In [1]:
model.score(X_test, y_test['Survived'])